# Self-Attention与Bert

BERT模型是Google在2019年提出的模型，一经提出就刷了几乎所有榜单的SOTA，可以说是NLP发展上里程碑级别的模型。同时也让原本一步到位的训练变成了现在pre-train，fine-tune两部分的训练。pre-train部分消耗的计算资源非常庞大，往往由大公司/高校来做，而我们要做的往往只是在已经预训练好的模型上，进行下游任务的fine-tune即可。

$\color{red}{警告：}$运行Bert-base的训练大约需要8GB的内存/显存，不能保证大家的电脑都能跑的起来，如果电脑配置不够且不要直接运行网上下载下来的Bert训练代码，容易发生OOM。  
本notebook的代码因为数据规模很小，经测试大约占用2GB的内存/显存，大部分同学的电脑应该都还跑得动。

In [1]:
import torch
from torch import nn
import numpy as np
import math
from tqdm import tqdm
from models.BertBasicModel import BertModel, BertConfig

你可以把Bert理解成一个性能非常好的，会根据上下文内容自动调整的Embedding。我们这里以Bert-base为例，它会为每个字输出一个768维的向量，这个向量不单单是这个词自己的向量，还融合了上下文信息。关于预训练的Mask方法这里就不涉及了，网上有海量的Bert分析文章。也可以查看`models.BertBasciModel`里的`BertForPreTraining`类。

由于模型很大这里没法给大家直接传到Github上，需要大家自己下载。如果大家想自己试试这个notebook，可以下载来试试。  
下载的Bert模型（pytorch版）一般会有三个文件：  
1. bert_config.json —— 存储这个Bert模型的各种超参数，例如dropout，hidden_size，多头注意力的头数等；
2. pytorch_model.bin —— 存储Bert模型的各种训练参数；
3. vocab.txt —— 词表；

其中最大的自然是pytorch_model.bin了。大小大概是400MB左右。

In [2]:
# bert的路径，各位自行修改，这个是google的标准中文Bert-base
bert_path = 'chinese_L-12_H-768_A-12'

Bert需要Bert对应的Tokenizer。这个Tokenizer需要Bert模型所在的目录路径或者vocab.txt的路径。

In [3]:
from utils.tokenizer import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(bert_path)

有了Tokenizer，我们就可以处理数据了。

In [4]:
from utils.apply_text_norm import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Bert的输入需要以下这些Tensor
# input_ids是转化为ids的文本输入
# input_mask是哪些地方是有文本的
# segment_ids是哪些地方是第一句，哪些地方是第二句
# label_id是转化为ids的标签输入
class InputFeature(object):
    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id

# 我们使用一个函数让我们之前的那个data_example转化为Bert版本的InputFeature
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in tqdm(enumerate(examples)):
        tokens_a = tokenizer.tokenize(process_sent(example.text))
        tokens_b = None
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]
            
        # Bert的特殊token，[CLS]和[SEP]
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        # segment_ids是用来指示第一句还是第二句的
        # 0是第一句，1是第二句
        # 我们这种只有第一句的任务的话其实就是全0的Tensor
        segment_ids = [0] * len(tokens)
        # 转换成ids
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        # 目前这些长度都是要输入的，所以input_mask全是1
        input_mask = [1] * len(input_ids)

        # 用0填充剩下的位置
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        if len(example.label) > max_seq_length - 2:
            example.label = example.label[: (max_seq_length - 2)]

        label_id = [label_map["B"]] + [label_map[tmp] for tmp in example.label]
        label_id += (len(input_ids) - len(label_id)) * [label_map["B"]]

        features.append(InputFeature(input_ids=input_ids,
                                     input_mask=input_mask, 
                                     segment_ids=segment_ids,
                                     label_id=label_id))
    return features

In [5]:
# 我们还是来做一下左传的CWS吧，这样不用再写一遍dataset_loader了（其实就是我懒）
from dataset_readers.cws import *
# 实例化一个readers
data_processor = Zuozhuan_Cws()
# 指定max_seq_length为128
max_seq_length = 128
# 指定batch_size为32
batch_size = 32
# 标签一共有两种（B和I）
num_labels = 2

# 获取labels
label_list = data_processor.get_labels()

# 获取训练语料
train_examples = data_processor.get_train_examples()
# 获取测试语料
dev_examples = data_processor.get_dev_examples()

print("Train_examples: ", len(train_examples))
print("Dev_examples: ", len(dev_examples))

def generate_data(examples, set_type="train"):
    print(set_type + " examples")
    for i in range(min(len(examples), 3)):
        print(examples[i].text)
        print(examples[i].label)
    sys.stdout.flush()
    features = convert_examples_to_features(examples, label_list, max_seq_length, tokenizer)
    input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    data = TensorDataset(input_ids, segment_ids, input_mask, label_ids)
    sampler = RandomSampler(data)
    return data, sampler

# convert data example into features
train_data, train_sampler = generate_data(train_examples, "train")
dev_data, dev_sampler = generate_data(dev_examples, "dev")

train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=batch_size)

Train_examples:  6083
Dev_examples:  1107
train examples
春秋左传定公
['B', 'I', 'B', 'I', 'B', 'I']
元年
['B', 'I']
春
['B']


6083it [00:00, 23638.04it/s]


dev examples
春秋左传隐公
['B', 'I', 'B', 'I', 'B', 'I']
惠公元妃孟子
['B', 'I', 'B', 'I', 'B', 'I']
孟子卒
['B', 'I', 'B']


1107it [00:00, 23110.44it/s]


In [6]:
# 指定训练5个epoch
# 因为Bert的优化器比较复杂，使用了Warmup策略，所以需要事先知道要跑多少步
num_train_epochs = 5
# 估算出一共要多少步
num_train_steps = int(math.ceil(len(train_examples) / batch_size) * num_train_epochs)
print("Total train steps: ", num_train_steps)

Total train steps:  955


我们要做的，就是读进来这个Bert模型，然后在它的基础上加几个我们需要的层。比如序列标注，一般是在最后加FC层和CRF层；分类问题，一般只需要FC层。  
这里以一个序列标注为例（为了你们的大作业着想），但是只加了一个FC层，没有加CRF层，保留一些实现的空间给大家~  
需要的bert_model_path就是Bert的目录位置。

In [7]:
class BertTagger(nn.Module):
    # 接收几个参数：
    # bert_model_path: bert的预训练文件的位置
    # bert_frozen: 是否冻结bert，让bert不进行训练
    # num_labels: 输出的标签个数
    def __init__(self, bert_model_path, bert_frozen=True, num_labels=4):
        super().__init__()
        self.num_labels = num_labels
        # 我们必须要一个config来初始化Bert，也就是之前下载下来的文件里的bert_config.json
        CONFIG_NAME = 'bert_config.json'
        config_path = os.path.join(bert_model_path, CONFIG_NAME)
        # 将这个json文件转化为一个BertConfig实例
        bert_config = BertConfig.from_json_file(config_path)
        # BertModel继承于PreTrainedBertModel类，这个类里实现了from_pretrained功能
        self.bert = BertModel.from_pretrained(bert_model_path)
        
        # 如果bert_frozen为true，就让所有bert里的参数都不需要记录梯度
        # 为了效率我们这里就冻结掉Bert的参数吧
        if bert_frozen:
            print("Please notice that the bert grad is false.")
            for param in self.bert.parameters():
                param.requires_grad = False
        
        # 记录下hidden_size，也就是最终的输出的维度
        self.hidden_size = bert_config.hidden_size
        # dropout率也沿用之前bert里的dropout率
        self.dropout = nn.Dropout(bert_config.hidden_dropout_prob)
        # 输出层，简单的一个线性层，从hidden_size映射到num_labels
        self.classifier = nn.Linear(self.hidden_size, self.num_labels) 

    # 我们输入的一般是(input_ids, segment_ids, input_mask, label_ids)
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        last_bert_layer, pooled_output = self.bert(input_ids, token_type_ids, attention_mask,
                                                   output_all_encoded_layers=False)
        last_bert_layer = last_bert_layer.view(-1, self.hidden_size)
        last_bert_layer = self.dropout(last_bert_layer)
        logits = self.classifier(last_bert_layer)
        # (batch_size, seq_length, num_labels)

        if labels is not None:
            # 带mask的交叉熵
            loss_fct = nn.CrossEntropyLoss()
            # 我们知道，实际上很多句子根本没有到seq_length那么长，那么多余的位置我们应该让模型预测什么呢？
            # 所以这里使用了attention_mask，其实也就是input_mask，来确定我们关心哪些位置
            # 我们只需要考虑attention_mask = 1 位置上的loss，其他地方模型预测什么，我们不关心
            active_loss = (attention_mask.view(-1) == 1)
            active_logits = logits.view(-1, self.num_labels)[active_loss]
            active_label = labels.view(-1)[active_loss]
            loss = loss_fct(active_logits, active_label)
            return loss
        else:
            # 直接返回logits
            return logits


model = BertTagger(bert_path, num_labels=num_labels)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
model.to(device)

Please notice that the bert grad is false.


BertTagger(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (

获取训练的数据集，同时得到了word_cnt之后就能初始化网络了。

我们来准备Bert专用的Adam优化器，这个优化器相比普通的Adam，加入了梯度修剪和warmup功能。

In [8]:
from utils.optimization import BertAdam
# 梯度裁剪，超过这个数字会被强制裁剪到这个数字
clip_grad = 1.0
# warmup率，前多少比例的步，lr逐渐升高
warmup_proportion = 0.1
# 学习率
learning_rate = 5e-5

# prepare optimizer
param_optimizer = list(model.named_parameters())
# bias和LayerNorm的参数不需要权重衰减，不然可能LayerNorm效果会不对
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {"params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], "weight_decay": 0.01},
    {"params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], "weight_decay": 0.0}]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=learning_rate,
                     warmup=warmup_proportion,
                     t_total=num_train_steps,
                     max_grad_norm=clip_grad)
print(optimizer)

BertAdam (
Parameter Group 0
    b1: 0.9
    b2: 0.999
    e: 1e-06
    lr: 5e-05
    max_grad_norm: 1.0
    schedule: warmup_linear
    t_total: 955
    warmup: 0.1
    weight_decay: 0.01

Parameter Group 1
    b1: 0.9
    b2: 0.999
    e: 1e-06
    lr: 5e-05
    max_grad_norm: 1.0
    schedule: warmup_linear
    t_total: 955
    warmup: 0.1
    weight_decay: 0.0
)


开始训练。一个epoch昨天看了看大概是40s，如果开着直播可能还会更慢点，可能1min一个epoch？

In [ ]:
import time
from sklearn.metrics import f1_score

def train(model, optimizer, train_dataloader, epoch=5):
    total_start_time = time.time()
    for i in range(epoch):
        epoch_start_time = time.time()
        print("epoch %d/%d" % (i + 1, epoch))
        model.train()
        total_loss = []
        for batch in train_dataloader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, segment_ids, input_mask, label_ids = batch
            optimizer.zero_grad()
            loss = model(input_ids, segment_ids, input_mask, label_ids)
            total_loss.append(loss.item())
            loss.backward()
            optimizer.step()
        print("loss: %.6f" % (sum(total_loss) / len(total_loss)))
        epoch_end_time = time.time()
        
        model.eval()
        total_gold = []
        total_pred = []
        for batch in dev_dataloader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, segment_ids, input_mask, label_ids = batch
            with torch.no_grad():
                logits = model(input_ids, segment_ids, input_mask)
                # 直接在最后一维上找哪个最大
                logits = torch.argmax(logits, dim=-1)
                active_loss = (input_mask.view(-1) == 1)
                active_logits = logits.view(-1)[active_loss]
                active_label = label_ids.view(-1)[active_loss]
                total_gold += active_label.detach().cpu().numpy().tolist()
                total_pred += active_logits.detach().cpu().numpy().tolist()
        print("f1 score: %.4f" % (f1_score(total_gold, total_pred)))
        
        print("epoch time: %d s" % (epoch_end_time - epoch_start_time))
    total_end_time = time.time()
    print("total time: %d s" % (total_end_time - total_start_time))

train(model, optimizer, train_dataloader, num_train_epochs)

epoch 1/5
loss: 0.356978
f1 score: 0.6367
epoch time: 36 s
epoch 2/5
loss: 0.218270
f1 score: 0.7343
epoch time: 37 s
epoch 3/5
loss: 0.204123
f1 score: 0.7534
epoch time: 37 s
epoch 4/5
loss: 0.198365
f1 score: 0.7607
epoch time: 37 s
epoch 5/5
